#Download the dataset and setup the environment
The repo is the clone of the dataset provided in the assignment.


In [ ]:
!git clone https://github.com/josedallatorre/DL-assignement.git

The setup file is the one given in the materials folder.

In [ ]:
%run /content/DL-assignement/cnw/setup.py

In [37]:
import csv
import glob
path = "/data/bboxes/CropOrWeed2/*.csv"
for fname in glob.glob(path):
  print(fname)
  with open(fname, newline='') as csvfile:
    reader = csvfile.readlines()
    for row in reader:
      words = row.split(',')
      if(int(words[4])==1):
        print("crop i guess "+str(words[4]))
      else:
        print("other i guess "+str(words[4]))

Output streaming troncato alle ultime 5000 righe.
/data/bboxes/CropOrWeed2/vwg-0641-0003.csv
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
other i guess0
other i guess0
other i guess0
other i guess0
other i guess0
crop i guess1
/data/bboxes/CropOrWeed2/ave-0256-0017.csv
other i guess0
/data/bboxes/CropOrWeed2/ave-0270-0021.csv
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
crop i guess1
cro

#Training phase

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, file_path, transform=None):
        self.file_path = file_path
        self.transform = transform
        self.image_names = self.load_image_names()

    def load_image_names(self):
        with open(self.file_path, 'r') as file:
            image_names = file.read().splitlines()
        return image_names

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = f'/data/images/{img_name}.jpg'
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

file_path = '/content/DL-assignement/train_split.txt'
transform = transforms.Compose([transforms.Resize((256, 256)),
                                transforms.ToTensor()])

dataset = CustomDataset(file_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

file_path_test = '/content/DL-assignement/test_split.txt'
transform = transforms.Compose([transforms.Resize((256, 256)),
                                transforms.ToTensor()])

testdataset = CustomDataset(file_path_test, transform=transform)
testloader = DataLoader(dataset, batch_size=32, shuffle=True)

#Create CNN
CNN

In [23]:


#Define a simple neural network as an example
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(64 * 256 * 256, 10)  # Adjust the output size based on your task

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
# Initialize the model, loss function, and optimizer
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for i, batch in enumerate(dataloader):
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch)

        # Dummy labels (replace with your actual labels)
        labels = torch.randint(0, 10, (batch.size(0),))

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print the loss for every 10 batches
        if i % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(dataloader)}], Loss: {loss.item():.4f}')


KeyboardInterrupt: ignored

#Evaluation phase
Validation

In [ ]:
# Validation loop
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0

with torch.no_grad():
  for val_batch in dataloader:
    val_outputs = model(val_batch)
    _, val_preds = torch.max(val_outputs, 1)

    # Dummy validation labels (replace with your actual validation labels)
    val_labels = torch.randint(0, 10, (val_batch.size(0),))
    total_correct += (val_preds == val_labels).sum().item()
    total_samples += val_batch.size(0)

    accuracy = total_correct / total_samples
    print(f'Validation - Epoch [{epoch + 1}/{num_epochs}], Accuracy: {accuracy:.4f}')

In [17]:
names = ['Weed', 'Crop']


In [18]:
img_path = '/data/images//vwg-0088-0005.jpg'
img = Image.open(img_path)
outputs = model.predict(img, conf=.1)[0]

AttributeError: ignored

In [ ]:
xyxys= []
confidences = []
classes = []
for result in outputs:
  boxes = result.boxes.cpu().numpy()
  xyxys.append(boxes.xyxy)
  confidences.append(boxes.conf)
  classes.append(boxes.cls)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
# print(img.shape)
for xy_ in xyxys:
  for idx , xy in enumerate(xy_):
    cv2.putText(img, text=str(confidences[0][idx]), org=(int(xy[0]) + 10,int(xy[1])+20), fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=.5, color=(0,255,0), thickness=1)
    cv2.putText(img, text=names[int(classes[0][idx])], org=(int(xy[0]) + 10,int(xy[1])+35), fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=.5, color=(0,255,0), thickness=1)
    cv2.rectangle(img, (int(xy[0]), int(xy[1])), (int(xy[2]), int(xy[3])), (0, 255, 0, 255), 2)
cv2_imshow(img)

In [22]:
# Evaluate the model
from sklearn.metrics import accuracy_score
import torch

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in testloader:
        preds = model(images)
        all_preds.extend(torch.argmax(preds, dim=1).numpy())
        all_labels.extend(labels.numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# Make predictions on a few sample images
sample_images, sample_labels = next(iter(testloader))
sample_preds = model(sample_images)
sample_preds_labels = torch.argmax(sample_preds, dim=1).numpy()

print("Predicted labels:", sample_preds_labels)
print("True labels:", sample_labels.numpy())

imshow(torchvision.utils.make_grid(sample_images))

ValueError: ignored

#seconda parte
paso

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have a custom dataset class, adjust as needed
class PlantDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        if self.transform:
            x = self.transform(x)

        return x, y

# Load and preprocess your dataset, assuming X contains images and y contains labels (0 for crops, 1 for weeds)
# Modify the path and preprocessing according to your dataset
X_train, X_val, y_train, y_val = train_test_split('content/DL-assignment/train_split')
train_dataset = PlantDataset(X_train, y_train, transform=transforms.Compose([...]))
val_dataset = PlantDataset(X_val, y_val, transform=transforms.Compose([...]))

# Assuming you have a DataLoader for your dataset
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class PlantCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(PlantCNN, self).__init__()
        self.resnet = resnet18(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Initialize the model, loss function, and optimizer
model = PlantCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {accuracy:.4f}')

# Save the model if needed
# torch.save(model.state_dict(), 'plant_cnn_model.pth')


ValueError: ignored

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Define your dataset paths
train_images_path = "/data/images"
train_annotations_path = "/content/DL-assignement/test_split.txt"
test_images_path = "/data/bboxes/CropOrWeed2"
test_annotations_path = "/content/DL-assignement/test_split.txt"

# Define some constants
input_size = (224, 224)  # Adjust based on your image size
num_classes = 1  # Assuming binary classification (object or not)

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(32 * (input_size[0] // 2) * (input_size[1] // 2), num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Create an instance of the model
model = SimpleCNN()

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, images_path, annotations_path, transform=None):
        self.images_path = images_path
        self.annotations_path = annotations_path
        self.transform = transform

        # Your code to initialize the dataset goes here

    def __len__(self):
        # Your code to return the dataset size goes here
        return dataset_size

    def __getitem__(self, idx):
        # Your code to load and preprocess data for a single sample goes here
        return sample_data

# Create data transforms if needed
transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.ToTensor(),
])

# Create dataset instances
train_dataset = CustomDataset(train_images_path, train_annotations_path, transform=transform)
test_dataset = CustomDataset(test_images_path, test_annotations_path, transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, targets)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

# Save the trained model if needed
torch.save(model.state_dict(), 'simple_cnn_model.pth')
